In [8]:
import numpy as np
import os
from PIL import Image  #from套件import函式
import matplotlib.pyplot as plt
import math
import random
import string

#定義卷積函式
def conv(image,kernel):
    i_h , i_w = image.shape #取原始影像長寬係數而已
    h , w = kernel.shape
    #卷積後的影像
    new_h = i_h - h + 1
    new_w = i_w - w + 1
    new_image = np.zeros((new_h,new_w),dtype=np.float)
    # 進行卷積操作，矩陣對應元素值相乘
    for i in range(new_w):
        for j in range(new_h):
            new_image[i,j]=np.sum(image[i:i+h,j:j+w]*kernel)
    # 去掉矩陣乘法後的小於0的和大於255的原值，重置為0和255
    # 用clip函式處理矩陣的元素，使元素值處於（0，255）之間
    new_image=new_image.clip(0,255)
    # 將新影象各元素的值四捨五入，然後轉成8位無符號整型
    new_image=np.rint(new_image).astype('uint8')
    #print(new_image.shape)
    return new_image

def pooling(inputmap,poolsize,poolstride):
    #inputmap sizes
    inputmap = np.array(inputmap)
    in_row,in_col = np.shape(inputmap)
    #outputmap sizes
    out_row,out_col = int(np.ceil(in_row/poolstride)),int(np.ceil(in_col/poolstride))
    outputmap = np.zeros((out_row,out_col))
    #取餘數
    mod_row,mod_col = np.mod(in_row,poolstride),np.mod(in_col,poolstride)
    
    #padding(edge mode是把邊緣數值擴充)
    temp_map = np.lib.pad(inputmap,((0,poolsize-mod_row),(0,poolsize-mod_col)), 'edge')
    
    #max pooling
    for r in range(0,out_row):
        for c in range(0,out_col):
            x = r*poolstride
            y = c*poolstride
            poolfield = temp_map[x:x+poolsize,y:y+poolsize]
            poolout = np.max(poolfield)
            outputmap[r, c] = poolout
            outputmap=np.rint(outputmap).astype('uint8')
            
    return outputmap
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

class NN:
  def __init__(self, NI, NH, NO):
    # number of nodes in layers
    self.ni = NI + 1 # +1 for bias
    self.nh = NH
    self.no = NO
    
    # initialize node-activations
    self.ai, self.ah, self.ao = [],[], []
    self.ai = [1.0]*self.ni
    self.ah = [1.0]*self.nh
    self.ao = [1.0]*self.no

    # create node weight matrices
    self.wi = makeMatrix (self.ni, self.nh)
    self.wo = makeMatrix (self.nh, self.no)
    # initialize node weights to random vals
    randomizeMatrix ( self.wi, -0.2, 0.2 )
    randomizeMatrix ( self.wo, -2.0, 2.0 )
    # create last change in weights matrices for momentum
    self.ci = makeMatrix (self.ni, self.nh)
    self.co = makeMatrix (self.nh, self.no)
    
  def runNN (self, inputs):
    if len(inputs) != self.ni-1:
      print('incorrect number of inputs')
    
    for i in range(self.ni-1):
      self.ai[i] = inputs[i]
      
    for j in range(self.nh):
      sum = 0.0
      for i in range(self.ni):
        sum +=( self.ai[i] * self.wi[i][j] )
      self.ah[j] = sigmoid (sum)
    
    for k in range(self.no):
      sum = 0.0
      for j in range(self.nh):        
        sum +=( self.ah[j] * self.wo[j][k] )
      self.ao[k] = sigmoid (sum)
      
    return self.ao
      
      
  
  def backPropagate (self, targets, N, M):
    # http://www.youtube.com/watch?v=aVId8KMsdUU&feature=BFa&list=LLldMCkmXl4j9_v0HeKdNcRA
    
    # calc output deltas
    # we want to find the instantaneous rate of change of ( error with respect to weight from node j to node k)
    # output_delta is defined as an attribute of each ouput node. It is not the final rate we need.
    # To get the final rate we must multiply the delta by the activation of the hidden layer node in question.
    # This multiplication is done according to the chain rule as we are taking the derivative of the activation function
    # of the ouput node.
    # dE/dw[j][k] = (t[k] - ao[k]) * s'( SUM( w[j][k]*ah[j] ) ) * ah[j]
    output_deltas = [0.0] * self.no
    for k in range(self.no):
      error = targets[k] - self.ao[k]
      output_deltas[k] =  error * dsigmoid(self.ao[k]) 
   
    # update output weights
    for j in range(self.nh):
      for k in range(self.no):
        # output_deltas[k] * self.ah[j] is the full derivative of dError/dweight[j][k]
        change = output_deltas[k] * self.ah[j]
        self.wo[j][k] += N*change + M*self.co[j][k]
        self.co[j][k] = change

    # calc hidden deltas
    hidden_deltas = [0.0] * self.nh
    for j in range(self.nh):
      error = 0.0
      for k in range(self.no):
        error += output_deltas[k] * self.wo[j][k]
      hidden_deltas[j] = error * dsigmoid(self.ah[j])
    
    #update input weights
    for i in range (self.ni):
      for j in range (self.nh):
        change = hidden_deltas[j] * self.ai[i]
        #print 'activation',self.ai[i],'synapse',i,j,'change',change
        self.wi[i][j] += N*change + M*self.ci[i][j]
        self.ci[i][j] = change
        
    # calc combined error
    # 1/2 for differential convenience & **2 for modulus
    error = 0.0
    for k in range(len(targets)):
      error = 0.5 * (targets[k]-self.ao[k])**2
    return error
        
        
  def weights(self):
    print ('Input weights:')
    for i in range(self.ni):
      print (self.wi[i])
    #print
    print ('Output weights:')
    for j in range(self.nh):
      print (self.wo[j])
    #print ''
  
  def test(self, patterns):
    for p in patterns:
      inputs = p[0]
      print ('Inputs:', p[0], '-->', self.runNN(inputs), '\tTarget', p[1])
  
  def train (self, patterns, max_iterations = 1000, N=0.5, M=0.1):
    for i in range(max_iterations):
      for p in patterns:
        inputs = p[0]
        targets = p[1]
        self.runNN(inputs)
        error = self.backPropagate(targets, N, M)
      if i % 50 == 0:
        print ('Combined error', error)
    self.test(patterns)
    

def sigmoid (x):
  return math.tanh(x)
  
# the derivative of the sigmoid function in terms of output
# proof here: 
# http://www.math10.com/en/algebra/hyperbolic-functions/hyperbolic-functions.html
def dsigmoid (y):
  return 1 - y**2

def makeMatrix ( I, J, fill=0.0):
  m = []
  for i in range(I):
    m.append([fill]*J)
  return m
  
def randomizeMatrix ( matrix, a, b):
  for i in range ( len (matrix) ):
    for j in range ( len (matrix[0]) ):
      matrix[i][j] = random.uniform(a,b)
    
# this loop is for read each image in this folder,directory_name is the folder name with images than conv and pooling.
def read_dir(directory_name):
    
    for filename in os.listdir(directory_name):
        ap = []
        A = Image.open(directory_name + "/" + filename, 'r')
        a = np.array(A)
        output_path = "outpic/"
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        #a = np.array(A)
        sobel_x = np.array(([-1,-2,-1],
                            [ 0, 0, 0],
                            [ 1, 2, 1]))
        sobel_y = np.array(([-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]))
        sobel_xy= np.array(([-1,-1, 0],
                            [-1, 0, 1],
                            [ 0, 1, 1]))
        prewitt_x = np.array(([-1,-1,-1],
                              [ 0, 0, 0],
                              [ 1, 1, 1]))
        prewitt_y = np.array(([-1, 0, 1],
                              [-1, 0, 1],
                              [-1, 0, 1]))
        prewitt_xy= np.array(([-2,-1, 0],
                              [-1, 0, 1],
                              [ 0, 1, 2]))
        laplacian_1 = np.array(([ 0,-1, 0],
                                [-1, 4,-1],
                                [ 0,-1, 0]))
        laplacian_2 = np.array(([-1,-1,-1],
                                [-1, 8,-1],
                                [-1,-1,-1]))
        
        kernel_list = ("sobel_x","sobel_y","sobel_xy","prewitt_x","prewitt_y","prewitt_xy","laplacian_1","laplacian_2")
        pat = []
        print("feature maps")
        for w in kernel_list:
            print("starting %s:"%w)
            #print("R convolution")
            R = conv(a[ :, :, 0],eval(w))
            #print(type(R))
            R = pooling( R, 2, 2)
            #print(R)
            #print(type(R))
            #print("G convolution")
            G = conv(a[ :, :, 1],eval(w))
            #print(G)
            G = pooling( G, 2, 2)
            #print(G)
            #print("B convolution")
            B = conv(a[ :, :, 2],eval(w))
            B = pooling( B, 2, 2)
            #合併三個通道的結果
            I = np.stack(( R, G, B), axis = 2)
            I = rgb2gray(I)
            #plt.imshow(I, cmap = plt.get_cmap('gray'))
            #plt.show()
            I = I.reshape((1,-1))
            ap = ap.append(I)
            #print(ap)
            ap = ap.dtype('uint8')
            #ap = ap.tolist
            #print(I)
            #print(np.shape(ap)) #(1, 507)
            #Image.fromarray(I).save("%s//%s%sp.jpg" % (output_path, filename, w))
        #ap開始出動
        #(ap_num,) = np.shape(ap)
        #ap_num = int(ap_num)
        onehot = np.array([0,0,0,0,0,0,0,0,0,0])
        filename = os.path.splitext(filename)[0]
        filename = int(filename)
        onehot[filename]=1
        onehot = onehot.tolist        
        pat = [ap,onehot]
        
    print(pat)
    #myNN = NN ( 1352, 100, 10)
    #myNN.train(pat)
    #print(np.shape(pat))
    #print(len(ap))
    #for i in range(0 ,ap_num):

        

            
            
        

#避免當 Python 檔案（模組、module）被引用的時候，檔案內的每一行都會被 Python 直譯器讀取並執行
if __name__ == "__main__":
    #  Image.open(file, mode) 
    #A=Image.open("number/1.png")
    read_dir("number")

feature maps
starting sobel_x:


AttributeError: 'NoneType' object has no attribute 'dtype'